In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import time
from geopy.geocoders import Nominatim

In [ ]:
building_permit_raw = pd.read_csv('Building_Permits___Current.csv')
building_permit = building_permit_raw[['Application/Permit Number','Permit Type','Status','Category','Expiration Date','Location']]

In [ ]:
#select building that are under construction and have permit issued or finaled
building_permit = building_permit[building_permit['Permit Type'] == 'Construction']
bp_finaled = building_permit[building_permit['Status'] == 'Permit Finaled'] 
bp_issued = building_permit[building_permit['Status'] == 'Permit Issued']
bp = pd.concat([bp_finaled,bp_issued])

In [ ]:
#check NaN in dataframe
num_nan = bp.isnull().sum().sum()
#The number of NaN in dataframe is 44, which is smaller than 0.5% in the dataframe 
#So we can directly delete the row containing NaN
bp.dropna(inplace = True)
bp.reset_index(drop=True,inplace = True)
bp.head(10)

In [ ]:
#bp92
start = time.time()

zipcode_list_92 = []
year_list_92 = []
month_list_92 = []
for i in range(len(bp_92)):
    if '(' in bp_92['Location'][8500 + i]:
        lat_lon = bp_92['Location'][8500 + i].split('(')[-1].split(')')[0]
        geolocator = Nominatim(timeout=10)
        location = geolocator.reverse(lat_lon)
        zipcode = location.address.split(',')[-2].split()[0]
        zipcode_list_92.append(zipcode)
    else:
        address = bp_92['Location'][8500 + i].split('(')[-1].split(')')[0]
        geolocator = Nominatim(timeout=10)
        location = geolocator.geocode(address)
        zipcode = location.address.split(',')[-2].split()[0]
        zipcode_list_92.append(zipcode)
    
    year = bp_92['Expiration Date'][8500 + i].split('/')[-1]
    month = bp_92['Expiration Date'][8500 + i].split('/')[0]
    year_list_92.append(year)
    month_list_92.append(month)

bp_92['Zipcode'] = zipcode_list_92
bp_92['Expected Construct Year'] = year_list_92
bp_92['Expected Construct Month'] = month_list_92
    
bp_92.to_csv('bp_92.csv',index = False)
    
end = time.time()
run_time = end - start

print(run_time)

In [ ]:
#Generate zip code from latitude, longitude and address
start = time.time()

zipcode_list = []
year_list = []
month_list = []
for i in range(len(bp)):
    if '(' in bp['Location'][i]:
        lat_lon = bp['Location'][i].split('(')[-1].split(')')[0]
        geolocator = Nominatim(timeout=10)
        location = geolocator.reverse(lat_lon)
        zipcode = location.address.split(',')[-2].split()[0]
        zipcode_list.append(zipcode)
    else:
        address = bp['Location'][i].split('(')[-1].split(')')[0]
        geolocator = Nominatim(timeout=10)
        location = geolocator.geocode(address)
        zipcode = location.address.split(',')[-2].split()[0]
        zipcode_list.append(zipcode)
    
    year = bp['Expiration Date'][i].split('/')[-1]
    month = bp['Expiration Date'][i].split('/')[0]
    year_list.append(year)
    month_list.append(month)

bp['Zipcode'] = zipcode_list
bp['Expected Construct Year'] = year_list
bp['Expected Construct Month'] = month_list
    
bp.to_csv('bp.csv',index = False)
    
end = time.time()
run_time = end - start

print(run_time)